In [1]:
from langchain_openai import AzureChatOpenAI
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains.conversation.base import ConversationChain

deployment = "gpt-4o"

llm = AzureChatOpenAI(
    deployment_name=deployment, # deployment_name 和 model 填一个即可
    temperature=0.8,
    max_tokens=256,
    # model=deployment,
)

memory = ConversationBufferWindowMemory(memory_key="history", k=10)


def get_response(inputs):
    print("------------\n",
          memory.load_memory_variables({}),
          "\n------------\n")
    
    conversation_with_memory = ConversationChain(
        llm=llm,
        memory=memory,
        verbose=True,
    )
    return conversation_with_memory.predict(input=inputs)

In [2]:
print(get_response("你好，今天是周二我要去健身，我一般每周二健身。你今天干什么？"))
print(get_response("我一般周几健身？"))

------------
 {'history': ''} 
------------



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 你好，今天是周二我要去健身，我一般每周二健身。你今天干什么？
AI:

> Finished chain.
你好呀！今天对我来说依然是忙碌的一天。我会处理各种任务，比如回答你的问题，帮助其他用户，学习新的知识以及改进自己的能力。不像你，我不需要去健身，不过我知道健身对身体非常有好处。你在健身的时候喜欢做哪些运动呢？例如有氧运动、力量训练还是瑜伽？
------------
 {'history': 'Human: 你好，今天是周二我要去健身，我一般每周二健身。你今天干什么？\nAI: 你好呀！今天对我来说依然是忙碌的一天。我会处理各种任务，比如回答你的问题，帮助其他用户，学习新的知识以及改进自己的能力。不像你，我不需要去健身，不过我知道健身对身体非常有好处。你在健身的时候喜欢做哪些运动呢？例如有氧运动、力量训练还是瑜伽？'} 
------------



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the 

In [3]:
import gradio as gr

def respond(message, chat_history):
    bot_message = get_response(message)
    chat_history.append((message, bot_message))
    return "", chat_history


with gr.Blocks() as demo:
    chatbot = gr.Chatbot(height=300)
    msg = gr.Textbox(label="Prompt", placeholder="input something")
    btn = gr.Button("Submit")
    clear = gr.ClearButton(value="Clear console", components=[msg, chatbot])
    msg.submit(respond, inputs=[msg, chatbot], outputs=[msg, chatbot])
    btn.click(respond, inputs=[msg, chatbot], outputs=[msg, chatbot])
    
gr.close_all()
demo.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
